In [ ]:
import linkpred
import matplotlib.pyplot as plt
import networkx as nx
import csv
import random

In [ ]:
g = nx.Graph()

# Read data from CSV file
with open('C:/Users/Pierfrancesco/Desktop/Data Science and Business Informatics/Social Network Analysis/PROJECT/weighted_graph.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip header row
    for row in reader:
        source, target, weight = row
        g.add_edge(source, target, weight=float(weight))

len(g.edges())

725330

In [ ]:
g1 = g

In [ ]:
g1.remove_edges_from(nx.selfloop_edges(g1))

In [ ]:
# Split edges into training and test sets
edges = list(g1.edges())
random.shuffle(edges)
split_idx = int(len(edges) * 0.8)
train_edges = edges[:split_idx]
test_edges = edges[split_idx:]

# Create training graph
G_train = nx.Graph()
G_train.add_edges_from(train_edges)

# Ensure all nodes in test edges are present in the training graph
test_edges = [(u, v) for u, v in test_edges if G_train.has_node(u) and G_train.has_node(v)]
test_edges_set = set(test_edges)


In [ ]:
# Define function to get top predictions and evaluate precision, recall, and accuracy
def get_top_predictions(predictor, top_k):
    # Predict links using the predictor
    results = predictor.predict()

    # Get the top k predictions
    top_results = results.top(top_k)
    predicted_edges = set(top_results.keys())

    # Calculate true positives
    true_positives = len(predicted_edges.intersection(test_edges_set))

    # Calculate false positives
    false_positives = len(predicted_edges - test_edges_set)

    # Calculate false negatives
    false_negatives = len(test_edges_set - predicted_edges)

    # Calculate true negatives
    total_possible_edges = len(G_train.nodes()) * (len(G_train.nodes()) - 1) // 2
    true_negatives = total_possible_edges - len(G_train.edges()) - true_positives - false_positives - false_negatives

    # Calculate precision, recall, and accuracy
    precision = true_positives / top_k
    recall = true_positives / len(test_edges_set)
    accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives)

    return precision, recall, accuracy, top_results

In [ ]:
# Number of top predictions to consider
k = len(test_edges)

In [ ]:
# Common Neighbors
cn = linkpred.predictors.CommonNeighbours(G_train, excluded=G_train.edges())


In [ ]:
cn_precision, cn_recall, cn_accuracy, cn_top = get_top_predictions(cn, k)

In [ ]:
# Jaccard Coefficient
jc = linkpred.predictors.Jaccard(G_train, excluded=G_train.edges())


In [ ]:
jc_precision, jc_recall, jc_accuracy, jc_top = get_top_predictions(jc, k)

In [ ]:
# Adamic/Adar
aa = linkpred.predictors.AdamicAdar(G_train, excluded=G_train.edges())


In [ ]:
aa_precision, aa_recall, aa_accuracy, aa_top = get_top_predictions(aa, k)

In [ ]:
print("Common Neighbors - Precision: {:.4f}, Recall: {:.4f}, Accuracy: {:.4f}".format(cn_precision, cn_recall, cn_accuracy))
print("Adamic/Adar - Precision: {:.4f}, Recall: {:.4f}, Accuracy: {:.4f}".format(aa_precision, aa_recall, aa_accuracy))
print("Jaccard Coefficient - Precision: {:.4f}, Recall: {:.4f}, Accuracy: {:.4f}".format(jc_precision, jc_recall, jc_accuracy))

Common Neighbors - Precision: 0.2098, Recall: 0.2098, Accuracy: 0.9953
Adamic/Adar - Precision: 0.2264, Recall: 0.2264, Accuracy: 0.9954
Jaccard Coefficient - Precision: 0.2774, Recall: 0.2774, Accuracy: 0.9957


In [1]:
"""kz = linkpred.predictors.Katz(G_train, excluded=G_train.edges())
kz_precision, kz_recall, kz_accuracy, kz_top = get_top_predictions(kz, k)

print("Katz - Precision: {:.4f}, Recall: {:.4f}, Accuracy: {:.4f}".format(kz_precision, kz_recall, kz_accuracy))"""
#ERRORE LiBRERIA


'kz = linkpred.predictors.Katz(G_train, excluded=G_train.edges())\nkz_precision, kz_recall, kz_accuracy, kz_top = get_top_predictions(kz, k)\n\nprint("Katz - Precision: {:.4f}, Recall: {:.4f}, Accuracy: {:.4f}".format(kz_precision, kz_recall, kz_accuracy))'

In [ ]:
# SimRank
sr = linkpred.predictors.SimRank(G_train, excluded=G_train.edges())
sr_precision, sr_recall, sr_accuracy, sr_top = get_top_predictions(sr, k)
print("SimRank - Precision: {:.4f}, Recall: {:.4f}, Accuracy: {:.4f}".format(sr_precision, sr_recall, sr_accuracy))

SimRank - Precision: 0.0537, Recall: 0.0537, Accuracy: 0.9944


In [1]:
"""gd = linkpred.predictors.GraphDistance(G_train, excluded=G_train.edges())
gd_precision, gd_recall, gd_accuracy, gd_top = get_top_predictions(gd, k)

print("GraphDistance - Precision: {:.4f}, Recall: {:.4f}, Accuracy: {:.4f}".format(gd_precision, gd_recall, gd_accuracy))"""
#MEMORY ERROR

'gd = linkpred.predictors.GraphDistance(G_train, excluded=G_train.edges())\ngd_precision, gd_recall, gd_accuracy, gd_top = get_top_predictions(gd, k)\n\nprint("GraphDistance - Precision: {:.4f}, Recall: {:.4f}, Accuracy: {:.4f}".format(gd_precision, gd_recall, gd_accuracy))'

In [ ]:
gd = linkpred.predictors.GraphDistance(g1, excluded=g1.edges())
gd_results = gd.predict()

top = gd_results.top(5)
for edge, score in top.items():
    print(edge, score)

Kienan Auton - Hendrika Whellams 1.5
Viola Lafferty - Abel McClymont 1.2000000000000002
Vally Chidwick - Kort Cuffe 1.2000000000000002
Thane Greet - Niall McVeighty 1.2000000000000002
Tabbitha Gilogly - Idette Wyllcock 1.2000000000000002


In [ ]:

# Generate the random predictions
rnd = linkpred.predictors.Random(g1, excluded=g1.edges())
rnd_results = rnd.predict()
# Use the custom function to get the top 5 results
top = rnd_results.top(5)
for edge, score in top.items():
    print(edge, score)
#MEMORY ERROR

Lany - Bryan Ferry 0.9999999840992545
Water From Your Eyes - Anatol Villar 0.999999957155844
Rico Blanco - HaSeul 0.9999999533548837
The Blue Stones - Dianemarie Apperley 0.9999999441346203
Secos & Molhados - Marc Anthony 0.9999999003403092


: 